In [1]:
import matplotlib.pyplot as plt
import dedalus.public as d3
import logging
import numpy as np

from dedalus.tools import jacobi
from dedalus.core import operators
from scipy import optimize

logger = logging.getLogger(__name__)

In [2]:
# Parameters
Ny = 128
dtype = np.complex128

# Bases
coords = d3.CartesianCoordinates('y')
dist = d3.Distributor(coords, dtype=dtype)
ybasis = d3.ChebyshevT(coords['y'], size=Ny,dealias=1, bounds=(0, 2))

y, = dist.local_grids(ybasis)

dy = lambda A: d3.Differentiate(A, coords['y'])

# Fields
u = dist.Field(name='u', bases=(ybasis))
v = dist.Field(name='v', bases=(ybasis))
w = dist.Field(name='w', bases=(ybasis))

gradu = dist.Field(name='gradu', bases=(ybasis))
gradv = dist.Field(name='gradv', bases=(ybasis))
gradw = dist.Field(name='gradw', bases=(ybasis))

U = dist.Field(name='U',bases=(ybasis))
U['g'] = y*(2-y)
Uy = dy(U)
alpha = -1
beta = 0
p = dist.Field(name='p', bases=(ybasis))
tau_u_1 = dist.Field(name='tau_u_1')
tau_u_2 = dist.Field(name='tau_u_2')

tau_v_1 = dist.Field(name='tau_v_1')
tau_v_2 = dist.Field(name='tau_v_2')

tau_w_1 = dist.Field(name='tau_w_1')
tau_w_2 = dist.Field(name='tau_w_2')

tau_p = dist.Field(name='tau_p')

ybasis2 = ybasis.derivative_basis(2)

# # Substitutions
lift_basis = ybasis.derivative_basis(2)
lift = lambda A, n: d3.Lift(A, lift_basis, n)
Re = 1000

# # Problem
problem = d3.IVP([u,v,w,p, tau_u_1, tau_u_2,tau_v_1, tau_v_2,tau_w_1, tau_w_2], namespace=locals())
problem.add_equation("dt(u) + 1j*alpha*u*U + v*Uy - 1/Re*(dy(dy(u))-alpha**2*u-beta**2*u) + lift(tau_u_1,-1) + lift(tau_u_2,-2) + 1j*alpha*p = 0")
problem.add_equation("dt(v) + 1j*alpha*v*U - 1/Re*(dy(dy(v))-alpha**2*v-beta**2*v) + lift(tau_v_1,-1) + lift(tau_v_2,-2) + dy(p) = 0")
problem.add_equation("dt(w) + 1j*alpha*w*U - 1/Re*(dy(dy(w))-alpha**2*w-beta**2*w) + lift(tau_w_1,-1) + lift(tau_w_2,-2) + 1j*beta*p = 0")
problem.add_equation("1j*alpha*u + dy(v) + 1j*beta*w = 0")

problem.add_equation("u(y=0) = 0")
problem.add_equation("u(y=2) = 0")

problem.add_equation("v(y=0) = 0")
problem.add_equation("v(y=2) = 0")

problem.add_equation("w(y=0) = 0")
problem.add_equation("w(y=2) = 0")

## Test

In [3]:
a = np.random.rand(Ny)+1j*np.random.rand(Ny)
b = np.random.rand(Ny)+1j*np.random.rand(Ny)

In [4]:
u['c'] = a
# Solver
solver = problem.build_solver(d3.RK443)
solver.stop_iteration = 1000
timestep = 1e-3
try:
    logger.info('Starting main loop')
    while solver.proceed:
        solver.step(timestep)
except:
    logger.error('Exception raised, triggering end of main loop.')
    raise
finally:
    solver.log_stats()
Ga = u['c']

2023-05-02 13:36:41,877 subsystems 0/1 INFO :: Building subproblem matrices 1/1 (~100%) Elapsed: 0s, Remaining: 0s, Rate: 8.9e+00/s
2023-05-02 13:36:41,879 __main__ 0/1 INFO :: Starting main loop
2023-05-02 13:36:42,660 solvers 0/1 INFO :: Stop iteration reached.
2023-05-02 13:36:42,661 solvers 0/1 INFO :: Final iteration: 1000
2023-05-02 13:36:42,662 solvers 0/1 INFO :: Final sim time: 1.0000000000000007
2023-05-02 13:36:42,663 solvers 0/1 INFO :: Setup time (init - iter 0): 0.1342 sec
2023-05-02 13:36:42,663 solvers 0/1 INFO :: Warmup time (iter 0-10): 0.02013 sec
2023-05-02 13:36:42,664 solvers 0/1 INFO :: Run time (iter 10-end): 0.7606 sec
2023-05-02 13:36:42,665 solvers 0/1 INFO :: CPU time (iter 10-end): 0.0002113 cpu-hr
2023-05-02 13:36:42,666 solvers 0/1 INFO :: Speed: 2.697e+06 mode-stages/cpu-sec


In [5]:
solver.state_adj[0]['c'] = b

try:
    logger.info('Starting adjoint loop')
    while solver.iteration>0:
        solver.step_adjoint(timestep)
except:
    logger.error('Exception raised, triggering end of main loop.')
    raise
finally:
    solver.log_stats()
    
GTb = solver.state_adj[0]['c']

2023-05-02 13:36:42,675 __main__ 0/1 INFO :: Starting adjoint loop
2023-05-02 13:36:42,677 solvers 0/1 INFO :: Warning: Only works if F==0 and linear direct equation.
2023-05-02 13:36:46,074 solvers 0/1 INFO :: Final iteration: 0
2023-05-02 13:36:46,075 solvers 0/1 INFO :: Final sim time: -2.1510571102112408e-16
2023-05-02 13:36:46,076 solvers 0/1 INFO :: Setup time (init - iter 0): 0.1342 sec
2023-05-02 13:36:46,076 solvers 0/1 INFO :: Timings unavailable because warmup did not complete.


In [6]:
term1 = np.vdot(b,Ga)
term2 = np.vdot(GTb,a)

In [7]:
print(term1,term2)

(-0.21139460135487403-0.13320223208119897j) (-0.21139460135493626-0.133202232081364j)


In [8]:
print('Rel. error = ', np.abs(term1-term2)/np.abs(term1))

Rel. error =  7.058995511712877e-13
